# Group 12 - Strategy Implementation and Backtesting

## 0. Import packages

If there are any missing packages, you can install them by typing in the code shell:
```terminal
!pip install <package_name>
```
or
```terminal
%pip install <package_name>
```

In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np
import psycopg
import json
import pprint
import mplfinance as mpf

from typing import List
from matplotlib import pyplot as plt
from numpy.testing import assert_almost_equal, assert_equal

                                              0.0/108.7 kB ? eta -:--:--
     ----------------------                  61.4/108.7 kB 1.7 MB/s eta 0:00:01
     -------------------------------------- 108.7/108.7 kB 2.1 MB/s eta 0:00:00
                                              0.0/64.9 kB ? eta -:--:--
     ---------------------------------------- 64.9/64.9 kB ? eta 0:00:00
                                              0.0/3.0 MB ? eta -:--:--
     ---                                      0.3/3.0 MB 5.4 MB/s eta 0:00:01
     --------                                 0.6/3.0 MB 8.0 MB/s eta 0:00:01
     ------------------                       1.4/3.0 MB 10.1 MB/s eta 0:00:01
     ---------------------------------------  3.0/3.0 MB 17.5 MB/s eta 0:00:01
     ---------------------------------------- 3.0/3.0 MB 16.1 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.41 requires requests_mock, which is not installed.
transformers 2.1.1 requires sentencepiece, which is not installed.
conda-repo-cli 1.0.41 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.41 requires nbformat==5.4.0, but you have nbformat 5.7.0 which is incompatible.
conda-repo-cli 1.0.41 requires requests==2.28.1, but you have requests 2.32.3 which is incompatible.


## 1. Data Preparation
### 1.1. Data Importing
First, we load the data from `yfinance`:

In [ ]:
# Load data
with open('database.json', 'rb') as fb:
    db_info = json.load(fb)
conn = psycopg.connect(
    host=db_info['host'],
    port=db_info['port'],
    dbname=db_info['database'],
    user=db_info['user'],
    password=db_info['password']
)
with psycopg.connect(
    host=db_info['host'],
    port=db_info['port'],
    dbname=db_info['database'],
    user=db_info['user'],
    password=db_info['password']
) as conn:
    # Open a cursor to perform database operations
    with conn.cursor() as in_sample_data:
        
        # Execute a query
        in_sample_data.execute("""
            SELECT m.datetime, m.tickersymbol, m.price, v.quantity
            FROM "quote"."matched" m
            LEFT JOIN "quote"."total" v
            ON m.tickersymbol = v.tickersymbol
            and m.datetime =  v.datetime
            where m.datetime between '2023-10-23' and '2023-11-23'
            and m.tickersymbol = 'VN30F2311'
        """)

        # Use fetchall() to get all the data of the query.
        # Note: fetchall() can be costly and inefficient.
        # Other efficient ways have been discussed extensively on the Internet. Or you can ask ChatGPT ;)
        in_sample_dataset = in_sample_data.fetchall()

        # Print the total number of ticks of that day
        print(f'Total number of tick: {len(in_sample_dataset)}')

        # Pretty print the first five ticks

    with conn.cursor() as out_sample_data:
        
        # Execute a query
        out_sample_data.execute("""
            SELECT m.datetime, m.tickersymbol, m.price, v.quantity
            FROM "quote"."matched" m
            LEFT JOIN "quote"."total" v
            ON m.tickersymbol = v.tickersymbol
            and m.datetime =  v.datetime
            where m.datetime between '2023-10-23' and '2023-12-3'
            and m.tickersymbol = 'VN30F2311'
        """)

        # Use fetchall() to get all the data of the query.
        # Note: fetchall() can be costly and inefficient.
        # Other efficient ways have been discussed extensively on the Internet. Or you can ask ChatGPT ;)
        out_sample_dataset = out_sample_data.fetchall()

        # Print the total number of ticks of that day
        print(f'Total number of tick: {len(out_sample_dataset)}')

In [ ]:
# Chuyển dữ liệu sang DataFrame của in_sample_data
in_sample_candle = pd.DataFrame(in_sample_dataset, columns=['datetime', 'tickersymbol', 'price', 'quantity'])
in_sample_candle['price'] = pd.to_numeric(in_sample_candle['price'], errors='coerce')
print(in_sample_candle['price'].dtype)
# Chuyển đổi cột datetime về kiểu datetime
in_sample_candle['datetime'] = pd.to_datetime(in_sample_candle['datetime'])
in_sample_candle.set_index('datetime', inplace=True)

# Resample dữ liệu theo khung 1 giờ (1h)
in_sample_candle_ohlc = in_sample_candle['price'].resample('1h').ohlc()
in_sample_candle_volume = in_sample_candle['quantity'].resample('1h').sum()

# Gộp dữ liệu OHLC với volume
in_sample_candle_ohlc['volume'] = in_sample_candle_volume

# Vẽ biểu đồ nến
mpf.plot(in_sample_candle_ohlc, type='candle', volume=True, style='charles',
        title=" In sample data VN30F2311 Candlestick Chart (1h)", ylabel="Price", ylabel_lower="Volume")

# Chuyển dữ liệu sang DataFrame của in_sample_data
out_sample_candle = pd.DataFrame(out_sample_dataset, columns=['datetime', 'tickersymbol', 'price', 'quantity'])
out_sample_candle['price'] = pd.to_numeric(out_sample_candle['price'], errors='coerce')
print(out_sample_candle['price'].dtype)
# Chuyển đổi cột datetime về kiểu datetime
out_sample_candle['datetime'] = pd.to_datetime(out_sample_candle['datetime'])
out_sample_candle.set_index('datetime', inplace=True)

# Resample dữ liệu theo khung 1 giờ (1h)
out_sample_candle_ohlc = out_sample_candle['price'].resample('1h').ohlc()
out_sample_candle_volume = out_sample_candle['quantity'].resample('1h').sum()

# Gộp dữ liệu OHLC với volume
out_sample_candle_ohlc['volume'] = out_sample_candle_volume

# Vẽ biểu đồ nến
mpf.plot(out_sample_candle_ohlc, type='candle', volume=True, style='charles',
        title=" Out sample data VN30F2311 Candlestick Chart (1h)", ylabel="Price", ylabel_lower="Volume")

## 2. Trading Algorithm

## 3. Backtesting

## 4. Evaluation